In [1]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

parser = argparse.ArgumentParser()
parser.add_argument('--image_folder', type=str, default='data/samples', help='path to dataset')
parser.add_argument('--config_path', type=str, default='config/yolov3.cfg', help='path to model config file')
parser.add_argument('--weights_path', type=str, default='weights/yolov3.weights', help='path to weights file')
parser.add_argument('--class_path', type=str, default='data/coco.names', help='path to class label file')
parser.add_argument('--conf_thres', type=float, default=0.8, help='object confidence threshold')
parser.add_argument('--nms_thres', type=float, default=0.4, help='iou thresshold for non-maximum suppression')
parser.add_argument('--batch_size', type=int, default=1, help='size of the batches')
parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
parser.add_argument('--img_size', type=int, default=416, help='size of each image dimension')
parser.add_argument('--use_cuda', type=bool, default=True, help='whether to use cuda if available')
opt = parser.parse_args(args=['--config_path', '/home/dllab/final/yolov3.cfg', '--class_path', '/home/dllab/final/clothes.names', '--image_folder', '/home/dllab/clothes/clothes_test/images', '--weights_path', '/tmp/work/50.pt', '--img_size', '640'])
print(opt)

cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs('output', exist_ok=True)

# Set up model
model = Darknet(opt.config_path, img_size=opt.img_size)
#model.load_weights(opt.weights_path)
model = torch.load(opt.weights_path)        #use this to load

if cuda:
    model.cuda()

model.eval() # Set in evaluation mode

dataloader = DataLoader(ImageFolder(opt.image_folder, img_size=opt.img_size),
                        batch_size=opt.batch_size, shuffle=False, num_workers=opt.n_cpu)

classes = load_classes(opt.class_path) # Extracts class labels from file

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

imgs = []           # Stores image paths
img_detections = [] # Stores detections for each image index

print ('\nPerforming object detection:')
prev_time = time.time()
for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
    # Configure input
    input_imgs = Variable(input_imgs.type(Tensor))
    
    input_imgs = input_imgs.to('cuda')

    # Get detections
    with torch.no_grad():
        detections = model(input_imgs)
        detections = non_max_suppression(detections, 80, opt.conf_thres, opt.nms_thres)


    # Log progress
    current_time = time.time()
    inference_time = datetime.timedelta(seconds=current_time - prev_time)
    prev_time = current_time
    print ('\t+ Batch %d, Inference Time: %s' % (batch_i, inference_time))

    # Save image and detections
    imgs.extend(img_paths)
    img_detections.extend(detections)

# Bounding-box colors
cmap = plt.get_cmap('tab20b')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]

print ('\nSaving images:')
# Iterate through images and save plot of detections
for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):

    print ("(%d) Image: '%s'" % (img_i, path))

    # Create plot
    img = np.array(Image.open(path))
    plt.figure()
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # The amount of padding that was added
    pad_x = max(img.shape[0] - img.shape[1], 0) * (opt.img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (opt.img_size / max(img.shape))
    # Image height and width after padding is removed
    unpad_h = opt.img_size - pad_y
    unpad_w = opt.img_size - pad_x

    # Draw bounding boxes and labels of detections
    if detections is not None:
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        bbox_colors = random.sample(colors, n_cls_preds)
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:

            print ('\t+ Label: %s, Conf: %.5f' % (classes[int(cls_pred)], cls_conf.item()))

            # Rescale coordinates to original dimensions
            box_h = ((y2 - y1) / unpad_h) * img.shape[0]
            box_w = ((x2 - x1) / unpad_w) * img.shape[1]
            y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
            x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]

            color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
            # Create a Rectangle patch
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2,
                                    edgecolor=color,
                                    facecolor='none')
            # Add the bbox to the plot
            ax.add_patch(bbox)
            # Add label
            plt.text(x1, y1, s=classes[int(cls_pred)], color='white', verticalalignment='top',
                    bbox={'color': color, 'pad': 0})

    # Save generated image with detections
    plt.axis('off')
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    plt.savefig('output/%d.png' % (img_i), bbox_inches='tight', pad_inches=0.0)
    plt.close()


Namespace(batch_size=1, class_path='/home/dllab/final/clothes.names', conf_thres=0.8, config_path='/home/dllab/final/yolov3.cfg', image_folder='/home/dllab/clothes/clothes_test/images', img_size=640, n_cpu=8, nms_thres=0.4, use_cuda=True, weights_path='/tmp/work/50.pt')


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))



Performing object detection:


/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  

	+ Batch 0, Inference Time: 0:00:01.410493
	+ Batch 1, Inference Time: 0:00:00.180351
	+ Batch 2, Inference Time: 0:00:00.035384
	+ Batch 3, Inference Time: 0:00:00.083473
	+ Batch 4, Inference Time: 0:00:00.095103
	+ Batch 5, Inference Time: 0:00:00.029654
	+ Batch 6, Inference Time: 0:00:00.083191
	+ Batch 7, Inference Time: 0:00:00.078892
	+ Batch 8, Inference Time: 0:00:00.035132
	+ Batch 9, Inference Time: 0:00:00.159502
	+ Batch 10, Inference Time: 0:00:00.036710
	+ Batch 11, Inference Time: 0:00:00.029672
	+ Batch 12, Inference Time: 0:00:00.067082
	+ Batch 13, Inference Time: 0:00:00.078730
	+ Batch 14, Inference Time: 0:00:00.031147
	+ Batch 15, Inference Time: 0:00:00.084519
	+ Batch 16, Inference Time: 0:00:00.089852
	+ Batch 17, Inference Time: 0:00:00.030428
	+ Batch 18, Inference Time: 0:00:00.080136
	+ Batch 19, Inference Time: 0:00:00.082823
	+ Batch 20, Inference Time: 0:00:00.034696
	+ Batch 21, Inference Time: 0:00:00.087003
	+ Batch 22, Inference Time: 0:00:00.08332

(17) Image: '/home/dllab/clothes/clothes_test/images/2018.jpg'
	+ Label: Light+LT, Conf: 0.98124
	+ Label: Light+ST, Conf: 0.83970
	+ Label: Light+ST, Conf: 0.91441
(18) Image: '/home/dllab/clothes/clothes_test/images/2019.jpg'
	+ Label: Light+ST, Conf: 1.00000
	+ Label: Deep+LU, Conf: 0.94885
	+ Label: Deep+ST, Conf: 0.99303
	+ Label: Deep+ST, Conf: 0.99881
(19) Image: '/home/dllab/clothes/clothes_test/images/2020.jpg'
	+ Label: Light+ST, Conf: 1.00000
	+ Label: Light+SU, Conf: 0.99872
	+ Label: Deep+SU, Conf: 1.00000
	+ Label: Deep+SU, Conf: 0.99522
	+ Label: Deep+LT, Conf: 0.82773


/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


(20) Image: '/home/dllab/clothes/clothes_test/images/2021.jpg'
	+ Label: Deep+LT, Conf: 0.99999
	+ Label: Light+ST, Conf: 0.90918
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Deep+ST, Conf: 0.99754
	+ Label: Deep+ST, Conf: 0.99997
(21) Image: '/home/dllab/clothes/clothes_test/images/2022.jpg'
	+ Label: Light+ST, Conf: 0.99991
	+ Label: Light+ST, Conf: 1.00000
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Deep+LU, Conf: 0.51222
	+ Label: Deep+LT, Conf: 0.66157
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 0.99145
	+ Label: Deep+ST, Conf: 1.00000
(22) Image: '/home/dllab/clothes/clothes_test/images/2023.jpg'
	+ Label: Light+LU, Conf: 0.97635
	+ Label: Light+LT, Conf: 0.34475
	+ Label: Light+ST, Conf: 0.95620
	+ Label: Deep+LU, Conf: 0.35940
	+ Label: Deep+SU, Conf: 0.96196
	+ Label: Deep+ST, Conf: 0.81334
(23) Image: '/home/dllab/clothes/clothes_test/images/2024.jpg'
	+ Label: Light+ST, Conf: 0.99984
	+ Label: Light

(50) Image: '/home/dllab/clothes/clothes_test/images/2051.jpg'
	+ Label: Deep+ST, Conf: 0.99932
	+ Label: Light+LT, Conf: 0.75913
	+ Label: Deep+LT, Conf: 0.07043
	+ Label: Light+ST, Conf: 0.98969
	+ Label: Light+ST, Conf: 0.99991
	+ Label: Light+ST, Conf: 0.81958
(51) Image: '/home/dllab/clothes/clothes_test/images/2052.jpg'
	+ Label: Deep+LT, Conf: 0.99392
	+ Label: Light+ST, Conf: 0.99659
	+ Label: Light+ST, Conf: 0.96496
	+ Label: Deep+ST, Conf: 0.98878
	+ Label: Deep+ST, Conf: 0.98788
	+ Label: Deep+ST, Conf: 0.99603
(52) Image: '/home/dllab/clothes/clothes_test/images/2053.jpg'
	+ Label: Light+SU, Conf: 0.77275
	+ Label: Light+LU, Conf: 0.41677
	+ Label: Light+ST, Conf: 1.00000
	+ Label: Light+LT, Conf: 0.73276
	+ Label: Deep+SU, Conf: 0.47221
(53) Image: '/home/dllab/clothes/clothes_test/images/2054.jpg'
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Light+SU, Conf: 0.98727
	+ Label: Deep+ST, Conf: 0.99997
	+ Label: Deep+SU, Conf: 0.91376
	+ Label: Deep+LT, Conf: 0.99982
(54) Image:

(91) Image: '/home/dllab/clothes/clothes_test/images/2092.jpg'
	+ Label: Deep+LU, Conf: 0.97519
	+ Label: Light+ST, Conf: 0.97697
	+ Label: Light+LT, Conf: 0.72848
	+ Label: Deep+SU, Conf: 0.33685
	+ Label: Deep+ST, Conf: 0.22748
(92) Image: '/home/dllab/clothes/clothes_test/images/2093.jpg'
(93) Image: '/home/dllab/clothes/clothes_test/images/2094.jpg'
	+ Label: Deep+LT, Conf: 0.98231
	+ Label: Deep+LU, Conf: 0.99974
	+ Label: Deep+LU, Conf: 0.87059
	+ Label: Deep+ST, Conf: 0.94510
(94) Image: '/home/dllab/clothes/clothes_test/images/2095.jpg'
	+ Label: Light+ST, Conf: 0.82454
	+ Label: Light+ST, Conf: 0.99988
	+ Label: Light+ST, Conf: 0.96985
	+ Label: Light+LT, Conf: 0.93718
(95) Image: '/home/dllab/clothes/clothes_test/images/2096.jpg'
	+ Label: Light+LT, Conf: 0.98530
	+ Label: Deep+LU, Conf: 0.98730
	+ Label: Deep+LU, Conf: 0.99993
	+ Label: Deep+ST, Conf: 0.99816
	+ Label: Deep+ST, Conf: 0.99442
	+ Label: Deep+ST, Conf: 0.98556
	+ Label: Deep+ST, Conf: 0.99995
	+ Label: Deep+ST,